In [ ]:
## imports
import neal
import dwavebinarycsp
import matplotlib.pyplot as plt
import networkx as nx

### Definições

In [ ]:
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['figure.dpi'] = 100 # 200 e.g. is really fine, but slower

def plot_color_map(sample, regions, neighbors, colors):
    G = nx.Graph()
    G.add_nodes_from(regions)
    G.add_edges_from(neighbors)
    
    color_map = {}
    for region in regions:
        for i in range(colors):
            if sample[f"{region}{i}"]:
                color_map[region] = i
                
    node_colors = [color_map.get(node) for node in G.nodes()]
    nx.draw_circular(G, with_labels=True, node_color=node_colors, node_size=3000, cmap=plt.cm.rainbow)
    plt.show()

def plot_map(regions, neighbors):
    G = nx.Graph()
    G.add_nodes_from(regions)
    G.add_edges_from(neighbors)
      
    nx.draw_circular(G, with_labels=True, node_size=3000, cmap=plt.cm.rainbow)
    plt.show()
    
def plot_hist(samples, variables=None):
    if variables is None: variables = list(samples.keys())
    results = {}
    for sample in samples:
        for key in sample:
            if key in results:
                results[key]+= sample[key]
            else:
                results[key] = sample[key]
                
    results = {key: ((results[key] / len(samples)) if key in results else 0.0) for key in sorted(variables)}

    plt.bar(results.keys(), results.values(), 0.6, color='b')
    plt.xticks(rotation=80)
    plt.show()

# Coloração do mapa do Brasil em regiões

<img src="regioes.jpg">

In [ ]:
regiões = ['Norte', 'Nordeste', 'Centro-Oeste', 'Sudeste', 'Sul']
vizinhos = {
    'Norte': ['Nordeste', 'Centro-Oeste'],
    'Nordeste': ['Norte', 'Centro-Oeste', 'Sudeste'],
    'Centro-Oeste': ['Norte', 'Nordeste', 'Sudeste', 'Sul'],
    'Sudeste' : ['Nordeste', 'Centro-Oeste', 'Sul'],
    'Sul' : ['Centro-Oeste', 'Sudeste']
}
pares = [(v, u) for v in vizinhos for u in vizinhos[v]]

plot_map(regiões, pares)

## Restrições

In [ ]:
# Cria-se um problema de satisfação de restrições (Binário)
psc = dwavebinarycsp.ConstraintSatisfactionProblem(dwavebinarycsp.BINARY)

In [ ]:
# Configurações válidas para que cada vértice selecione apenas uma cor.
configurações = {(0, 0, 0, 1), (0, 0, 1, 0), (0, 1, 0, 0), (1, 0, 0, 0)}
cores = len(configurações)

fixo = False

variáveis = set()

# Adiciona-se a restrição de que cada região selecione apenas uma cor.
for região in regiões:
    _variáveis = [f"{região}{i}" for i in range(cores)]
    if fixo and região == "Centro-Oeste":
        psc.add_constraint({(1, 0, 0, 0),},  _variáveis)
    else:
        psc.add_constraint(configurações, _variáveis)
    variáveis.update(_variáveis)

In [ ]:
# Vértices vizinhos não devem possuir a mesma cor.
def cores_distintas(v, u):
    return not (v and u)

In [ ]:
# Adiciona-se a restrição de que regiões vizinhas tenham cores diferentes.
for v, u in pares:
    for i in range(colors):
        _variáveis = [f"{v}{i}", f"{u}{i}"]
        psc.add_constraint(cores_distintas, _variáveis)
        variáveis.update(_variáveis)

## Conversão

In [ ]:
# Converte o problema de restrições binárias em um modelo binário quadrático.
mbq = dwavebinarycsp.stitch(psc)

## Resolução

In [ ]:
# Usa o simulador para resolver o problema
num = 10

sampler = neal.SimulatedAnnealingSampler() 
resposta = sampler.sample(mbq, num_reads=num)

In [ ]:
# Exibe a amostra de menor energia (se atender às restrições)
amostras = list(resposta.samples())

plot_hist(amostras, variáveis)

if not psc.check(amostras[0]): 
    print("Coloração Falhou.")
else:
    plot_color_map(amostra, regiões, pares, cores)